In [30]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [31]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
from elasticsearch import Elasticsearch

# test

In [33]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '4f49586bc1ea', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'V87AdtE5RC6hKQz_qQH18w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# adding data into Elasticsearch

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"
es_client.indices.create(index=index_name, body=index_settings)

In [35]:
from tqdm.auto import tqdm

In [36]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc) # using index to add data to elastic

100%|██████████| 948/948 [00:32<00:00, 29.42it/s]


# Seach engine

In [11]:
query = "I just join the course, Can I still join it?"

In [12]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"], # 3 is the boost factor.
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"        # similar to SQL's where stmt.
                }
            }
        }
    }
}

In [14]:
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits']

[{'_index': 'course_questions',
  '_id': 'FF6_cZABlWAZzpXF77j0',
  '_score': 90.17454,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course_questions',
  '_id': 'GV6_cZABlWAZzpXF8Lic',
  '_score': 54.057133,
  '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
   'section': 'General course-related questions',
   'question': 'Course - Can I follow the course after i

In [22]:
e_response = response['hits']['hits']

result_docs = []

for hit in e_response:
    result_docs.append(hit['_source']['_score'])

result_docs

[{'course': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'},
  'section': 90.17454},
 {'course': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
   'section': 'General course-related questions',
   'question': 'Course - Can I follow the course after it finishes?',
   'course': 'data-engineering-zoomcamp'},
  'section': 54.057133},
 {'course': {'text': 'You can start by installing and setting up all the

# -----------------function---------------------

In [37]:
def elasticsearch(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], # 4 is the boost factor.
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"        # similar to SQL's where stmt.
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    e_response = response['hits']['hits']

    result_docs = []

    for hit in e_response:
        combined_info = {
            'source' : hit['_source'],
            'score' : hit['_score']
        }

        result_docs.append(combined_info)
    return result_docs        

In [38]:
q = "How do I execute a command in a running docker container?"
elasticsearch(q)

[{'source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
   'section': 'Module 1: Docker and Terraform',
   'question': 'PGCLI - running in a Docker 